## this jupyter implements a simple operation using neural network, such as OR , AND, XOR.

- if you want OR operation, replace 12-th line with  `y[i]=[x1|x2]` , and change `is_xor=False`
- if you want AND operation, replace 12-th line with `y[i]=[x1&x2]` , and change `is_xor=False`
- if you want XOR operation, replace 12-th line with `y[i]=[x1^x2]` , and change `is_xor=True`

In [ ]:
from torch.autograd import  Variable
import numpy as np
import torch

is_xor = True
# step1 : generate the dataset for training
x=np.zeros([4,2])
y=np.zeros([4,1])
for i in range(0,4):
    x1=(int)(i/2)
    x2=i%2
    x[i]=np.array([x1,x2])
    y[i]=[x1^x2]
input_x=Variable(torch.Tensor(x))
input_x=input_x.float()
y=Variable(torch.from_numpy(y))
y=y.float()


# step2 : build the network

class network(torch.nn.Module):
    def __init__(self,xor=False):
        super(network,self).__init__()
        if xor:
            self.net = torch.nn.sequential(
                torch.nn.Linear(2,2),
                torch.nn.Sigmoid(),
                torch.nn.Linear(2,1),
                torch.nn.Sigmoid(),
            )
        else:
            self.net = torch.nn.Sequential(
                torch.nn.Linear(2,1)
                torch.nn.Sigmoid(),
            )
            
        
    def forward(self,input_x):
        return self.net(input_x)
    
net=network(xor=is_xor) 
loss_function=torch.nn.BCELoss() 
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1, momentum=0.9) # you can get better results with `momentum=0.9`

for i in range(10000): 
    out=net(input_x) 
    loss=loss_function(out,y) 
    print ("loss is %f"%loss.data.numpy()) 
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step()
    
out=net(input_x)
print(out)
print(y)

In [ ]:
import torch
import torch.nn as nn
import numpy as np

cuda_available=torch.cuda.is_available()
train_data = [[0,0],
              [0,1],
              [1,0],
              [1,1]]
train_label = [[0],[1],[1],[0]]

test_data = [[0,0],
              [0,1],
              [1,0],
              [1,1]]

train_data = torch.tensor(train_data, dtype=torch.float32)
train_label = torch.tensor(train_label, dtype=torch.float32)
test_data = torch.tensor(test_data, dtype=torch.float32)


# parameters
input_nums = 2
hidden_nums = 2
output_nums = 1
epochs = 10000
learning_rate = 0.1

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet,self).__init__()
        self.f1 = nn.Linear(input_nums, hidden_nums)
        self.f2 = nn.Linear(hidden_nums, output_nums)
#         self.weight_init(0.5,0.5)
            
    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            self._modules[m].weight.data.normal_(mean, std)
            self._modules[m].bias.data.zero_()
    
    def forward(self, x):
        result = torch.sigmoid(self.f1(x))
        result = torch.sigmoid(self.f2(result))
        return result


net = MyNet()
if cuda_available:
    net = net.cuda()
    train_data = train_data.cuda()
    train_label = train_label.cuda()
    test_data = test_data.cuda()

L2_loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate,)

for i in range(epochs):
    pred = net(train_data)
    optimizer.zero_grad()
    Loss = L2_loss_fn(pred,train_label)
    Loss.backward()
    optimizer.step()

    print("epoch is :%d , loss is: %.5f" % (i, Loss))

for data in test_data:
    pred = net(data)
    print("data is : ",data,"  ,pred_label is:",pred)
        
        
        